In [1]:
from ThuGEnv import ThuGEnv

env = ThuGEnv(
    attack="AAD_Comprise"
)

./data/tables/AADManagedIdentitySignInLogs.csv
./data/tables/AADNonInteractiveUserSignInLogs.csv
./data/tables/AADServicePrincipalSignInLogs.csv
./data/tables/AADSignInEventsBeta.csv
./data/tables/AADSpnSignInEventsBeta.csv
./data/tables/AuditLogs.csv
./data/tables/AzureActivity.csv
./data/tables/DeviceTvmInfoGatheringKB.csv
./data/tables/DeviceTvmSecureConfigurationAssessmentKB.csv
./data/tables/DeviceTvmSoftwareEvidenceBeta.csv
./data/tables/DeviceTvmSoftwareInventory.csv
./data/tables/DeviceTvmSoftwareVulnerabilities(10k).csv
./data/tables/DeviceTvmSoftwareVulnerabilities.csv
./data/tables/ExposureGraphEdges.csv
./data/tables/ExposureGraphNodes(10k).csv
./data/tables/Heartbeat.csv
./data/tables/IdentityInfo.csv
./data/tables/MicrosoftGraphActivityLogs.csv
./data/tables/SecurityEvent.csv
./data/tables/SigninLogs.csv
./data/tables/WindowsEvent.csv


In [2]:
env._all_questions

[{'problem': 'A potential malicious user is using Graph API to enumerate all users in the tenant. Find the client ID of the application that is doing this.',
  'answer': '1950a258-227b-4e31-a9cf-717495945fc2',
  'tags': ['NLtoKQL']},
 {'problem': 'In this incident, a credential is added to a applications that have the mail.read role. What is the name of the application?',
  'golden_query': 'AuditLogs \n| where TimeGenerated between(datetime(2024-06-24 17:42:13) .. datetime(2024-06-24 17:50:29))\n| where OperationName has_any ("Add service principal", "Certificates and secrets management")  \n| where Result =~ "success"  \n| where tostring(InitiatedBy.user.userPrincipalName) has "@" or tostring(InitiatedBy.app.displayName) has "@"  \n| mv-expand TargetResources \n| extend info = tostring(TargetResources.id) ',
  'answer': 'ARTBAS AAD App'}]

In [ ]:
action = """SELECT 
    TimeGenerated,
    OperationName,
    Result,
    InitiatedByUser.userPrincipalName,
    InitiatedByApp.displayName,
    TargetResources.id AS info
FROM 
    AuditLogs
CROSS APPLY 
    (SELECT TargetResources.id FROM TargetResources) AS TargetResourcesExpanded
WHERE 
    TimeGenerated BETWEEN '2024-06-24 17:42:13' AND '2024-06-24 17:50:29'
    AND (OperationName LIKE '%Add service principal%' OR OperationName LIKE '%Certificates and secrets management%')
    AND Result = 'success'
    AND (InitiatedByUser.userPrincipalName LIKE '%@%' OR InitiatedByApp.displayName LIKE '%@%')"""

In [5]:
question, info = env.reset(0)

In [6]:
question

'In this incident, a credential is added to a applications that have the mail.read role. What is the name of the application?'

In [ ]:
observation, reward, done, info = env.step(
    action=action
)

In [ ]:
observation, reward, done, info

('OperationalError: near "(": syntax error',
 0,
 False,
 {'success': False, 'submit': False})

In [ ]:
env.curr_trajectory

[{'action': "SELECT \n    TimeGenerated,\n    OperationName,\n    Result,\n    InitiatedByUser.userPrincipalName,\n    InitiatedByApp.displayName,\n    TargetResources.id AS info\nFROM \n    AuditLogs\nCROSS APPLY \n    (SELECT TargetResources.id FROM TargetResources) AS TargetResourcesExpanded\nWHERE \n    TimeGenerated BETWEEN '2024-06-24 17:42:13' AND '2024-06-24 17:50:29'\n    AND (OperationName LIKE '%Add service principal%' OR OperationName LIKE '%Certificates and secrets management%')\n    AND Result = 'success'\n    AND (InitiatedByUser.userPrincipalName LIKE '%@%' OR InitiatedByApp.displayName LIKE '%@%')",
  'observation': 'OperationalError: near "(": syntax error',
  'reward': 0,
  'done': False,
  'info': {'success': False, 'submit': False}}]

In [13]:
action = """SELECT *
FROM AuditLogs
WHERE TimeGenerated BETWEEN '2024-06-24 10:42:13' AND '2024-06-24 10:50:29'
  AND (
    LOWER(OperationName) LIKE '%add service principal%' OR
    LOWER(OperationName) LIKE '%certificates and secrets management%'
  )
  AND LOWER(Result) = 'success';
"""
# action = """PRAGMA table_info(AuditLogs);"""

observation, reward, done, info = env.step(
    action=action
)

observation, reward, done, info

([('2024-06-24 10:42:14.000000', '/tenants/3cd87a41-1f61-4aef-a212-cefdecd9a2d1/providers/Microsoft.aadiam', 'Update application – Certificates and secrets management ', 1.0, 'ApplicationManagement', None, 0, '31c37bc6-9f39-4dc9-bccd-24c8fc50a396', 'Microsoft.aadiam', 'Microsoft.aadiam', 4, '[{"key":"AppId","value":"dfbd6152-e2f7-483e-b775-7b915bdf02ed"}]', 'Directory_31c37bc6-9f39-4dc9-bccd-24c8fc50a396_I9C7O_245313131', '{"user":{"displayName":null,"id":"7b398c54-bcfb-4f9b-8f6f-c8d36ad914a4","userPrincipalName":"mmelendez@DefenderATEVET17.onmicrosoft.com","ipAddress":"71.231.82.253","roles":[]}}', 'Core Directory', 'success', '[{"id":"841dc6f7-70dd-4473-9249-5436b5e516d0","displayName":"ARTBAS AAD App","type":"Application","modifiedProperties":[{"displayName":"KeyDescriptio ... (5507 characters truncated) ... erify,DisplayName=]\\"]"},{"displayName":"Included Updated Properties","oldValue":null,"newValue":"\\"KeyDescription\\""}],"administrativeUnits":[]}]', '3cd87a41-1f61-4aef-a212-

In [ ]:
# WHERE TimeGenerated BETWEEN '2024-06-24 10:42:13' AND '2024-06-24 10:50:29'
#   AND (OperationName = 'Add service principal' OR OperationName = 'Certificates and secrets management')
#   AND Result = 'success'

In [2]:
import yaml
with open("./data/DefenderFullSchema.yml", "r") as f:
    schema = yaml.safe_load(f)

In [5]:
schema[0]

{'Name': 'AADSignInEventsBeta',
 'InternalName': 'AADSignInEventsBeta',
 'Description': "Information about Azure Active Directory (AAD) sign-in events either by a user (interactive) or a client on the user's behalf (non-interactive)",
 'Conditions': 'AadTablesCondition',
 'Columns': [{'Name': 'Timestamp',
   'Header': 'ColumnHeaderTimestamp',
   'Description': 'Date and time when the record was generated',
   'Type': 'DateTime?'},
  {'Name': 'Application',
   'Header': 'ColumnHeaderApplication',
   'Description': 'Application that performed the recorded action',
   'Type': 'string',
   'Entity': 'application'},
  {'Name': 'ApplicationId',
   'Header': 'ColumnHeaderApplicationId',
   'Description': 'Unique identifier for the application',
   'Type': 'string',
   'Entity': 'application',
   'Format': 'uuid'},
  {'Name': 'LogonType',
   'Header': 'ColumnHeaderLogonType',
   'Description': 'Type of logon session in a JSON array format e.g., \'["interactiveUser"]\' or \'["nonInteractiveUser